<a href="https://colab.research.google.com/github/lawliaet/ds/blob/main/Streamlit_Image_Classification_Web_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -U ipykernel==5.3.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 120 kB 4.8 MB/s 
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 5.3.4 which is incompatible.


In [3]:
!pip install -q streamlit

     |████████████████████████████████| 9.1 MB 4.8 MB/s 
     |████████████████████████████████| 4.3 MB 51.2 MB/s 
     |████████████████████████████████| 232 kB 67.0 MB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
     |████████████████████████████████| 164 kB 52.4 MB/s 
     |████████████████████████████████| 111 kB 66.7 MB/s 
     |████████████████████████████████| 181 kB 65.7 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 51 kB 8.6 MB/s 


In [4]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/-stablngroke-linux-amd64.zip

--2022-06-07 16:11:45--  https://bin.equinox.io/c/4VmDzA7iaHb/-stablngroke-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.161.241.46, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘-stablngroke-linux-amd64.zip’

-stablngroke-linux- 100%[===================>]  13.19M  5.39MB/s    in 2.4s    

2022-06-07 16:11:48 (5.39 MB/s) - ‘-stablngroke-linux-amd64.zip’ saved [13832437/13832437]



In [5]:
!unzip /content/-stablngroke-linux-amd64.zip

Archive:  /content/-stablngroke-linux-amd64.zip
  inflating: ngrok                   


In [6]:
!ls

drive  ngrok  sample_data  -stablngroke-linux-amd64.zip


In [7]:
!./ngrok 

NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostname=foo.com 443  # TLS traffic for foo.com to port 443
    ngrok start foo bar baz          # start tunnels from the configuration file

VERSI

In [8]:
!pip install pyngrok==4.1.8

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pyngrok: filename=pyngrok-4.1.8-py3-none-any.whl size=16348 sha256=06c7db837d50298d33eaefb19a7ee0b7d79559fdd391b2ae83b5683619794bd4
  Stored in directory: /root/.cache/pip/wheels/63/49/e3/03bfea32f8f48ae169091003ecd82f7ac6a598c9683326f995
Successfully built pyngrok


https://ngrok.com/

⚠️⚠️Update the token below from creating an account in ngrok website⚠️⚠️

You can ignore Ngrok step completely if you are not looking to expose your app to internet



In [ ]:
!./ngrok authtoken 21j68CC03JWN6TY2Vh9sqDABXib_2JYYHuEduf95MSPoDYz6z

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
!unzip /content/drive/MyDrive/chess_model.zip

Archive:  /content/drive/MyDrive/chess_model.zip
   creating: content/chess_model/
   creating: content/chess_model/variables/
  inflating: content/chess_model/variables/variables.data-00000-of-00001  
  inflating: content/chess_model/variables/variables.index  
  inflating: content/chess_model/saved_model.pb  
   creating: content/chess_model/assets/
  inflating: content/chess_model/keras_metadata.pb  


In [10]:
%%writefile score.py

import tensorflow as tf
import numpy as np
import streamlit as st
from PIL import Image
import requests
from io import BytesIO

st.set_option('deprecation.showfileUploaderEncoding', False)
st.title("Image Classifier")
st.text("Provide URL of flower Image for image classification")

@st.cache(allow_output_mutation=True)
def load_model():
  model = tf.keras.models.load_model('/content/content/chess_model')
  return model

with st.spinner('Loading Model Into Memory....'):
  model = load_model()

classes=['Bishop', 'King', 'Knight', 'Pawn', 'Queen', 'Rook']


def scale(image):
  image = tf.cast(image, tf.float32)
  image /= 255.0 # external data norm while training
  # in our case norm is part of sequential model

  return tf.image.resize(image,[224,224])

def decode_img(image):
  img = tf.image.decode_jpeg(image, channels=3)
  img = scale(img)
  return np.expand_dims(img, axis=0)

path = st.text_input('Enter Image URL to Classify.. ','https://storage.googleapis.com/download.tensorflow.org/example_images/592px-Red_sunflower.jpg')
image_file = st.file_uploader("Upload Images", type=["png","jpg","jpeg"])
if path is not None:
    content = requests.get(path).content

    st.write("Predicted Class :")
    with st.spinner('classifying.....'):
      label =np.argmax(model.predict(decode_img(content)),axis=1)
      st.write(np.array(classes))
      st.write(model.predict(decode_img(content)))
      st.write(classes[label[0]])    
    st.write("")
    image = Image.open(BytesIO(content))
    st.image(image, caption='Classifying Bean Image', use_column_width=True)    


Writing score.py


In [ ]:
# import tensorflow as tf
# model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/models/flower/models')

In [ ]:
!nohup streamlit run score.py >/dev/null &

nohup: redirecting stderr to stdout


In [ ]:
!streamlit run score.py & npx localtunnel --port 8501

2022-06-07 11:20:18.299 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
npx: installed 22 in 2.117s
your url is: https://warm-hats-remain-34-73-109-20.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.73.109.20:8501

2022-06-07 11:20:48.934774: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
  Stopping...
^C


In [ ]:
from pyngrok import ngrok

url = ngrok.connect(port=8503)
url

'http://b34b-34-73-109-20.ngrok.io'

In [ ]:
!cat nohup.out

cat: nohup.out: No such file or directory
